In [3]:
import numpy as np
import pandas as pd

"""
# Simple Moving Average
# https://school.stockcharts.com/doku.php?id=technical_indicators:moving_averages

Params: 
    data: pandas DataFrame
    period: smoothing period
    column: the name of the column with values for calculating EMA in the 'data' DataFrame
    
Returns:
    copy of 'data' DataFrame with 'ema[period]' column added
"""

def sma(df,period):
    # Copy Close Column
    df['SMA'] = df['CLOSE']
    # Turn SMA(Close_Values) Col to SMA
    df['SMA'] = df.rolling(window=5).mean()
    return df


"""
# Exponential Moving Average
# https://school.stockcharts.com/doku.php?id=technical_indicators:moving_averages

Params: 
    data: pandas DataFrame
    period: smoothing period
    column: the name of the column with values for calculating EMA in the 'data' DataFrame
    
Returns:
    copy of 'data' DataFrame with 'ema[period]' column added
"""
def ema(data,period):
    # select close column
    close_col=data['CLOSE']
    # ema formula
    data['ema'+ str(period)] = data.ewm(ignore_na=False,min_periods=period, span=period,adjust=True).mean()
    return data

df = pd.DataFrame({'CLOSE': [1, 1, 2, 3, 4, 5, 6, 10, 12, 14, 13, 12, 12]})
ema5=ema(df,5)
sma5=sma(df,5)
df.head(15)

,CLOSE,ema5,SMA
0,1,NaN,NaN
1,1,NaN,NaN
2,2,NaN,NaN
3,3,NaN,NaN
4,4,2.834123,2.2
5,5,3.625564,3.0
6,6,4.466246,4.0
7,10,6.385726,5.6
8,12,8.307131,7.4
9,14,10.238242,9.4
